In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import nltk
import keras
from nltk.corpus import stopwords


Using TensorFlow backend.


In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
df = pd.read_csv(r'C:\Users\Gundeep Gulati\Desktop\Language_Identification_dataset\dataset.csv')

In [5]:
df.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      22000 non-null  object
 1   language  22000 non-null  object
dtypes: object(2)
memory usage: 343.9+ KB


In [7]:
df['Text'] = df['Text'].str.lower()

In [8]:
df.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


In [9]:
max_words = 50000
max_len = 100


In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(list(df['Text']))
train_df = tokenizer.texts_to_sequences(list(df['Text']))

In [11]:
train_df = tf.keras.preprocessing.sequence.pad_sequences(train_df,maxlen = max_len)

In [12]:
max_len

100

In [13]:
train_df

array([[    0,     0,     0, ...,  2170,    82,  3638],
       [    0,     0,     0, ...,    16,  4068, 26452],
       [    0,     0,     0, ...,     0,     0, 16664],
       ...,
       [    0,     0,     0, ...,   118,  1435,   168],
       [    0,     0,     0, ...,    16,  2029,   471],
       [    0,     0,     0, ...,   178,  6523,     1]])

In [14]:
len(tokenizer.word_index)

273967

In [15]:
Y = df['language']
Y

0        Estonian
1         Swedish
2            Thai
3           Tamil
4           Dutch
           ...   
21995      French
21996        Thai
21997     Spanish
21998     Chinese
21999    Romanian
Name: language, Length: 22000, dtype: object

In [16]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
le = preprocessing.LabelEncoder()

In [17]:
le.fit(Y)

LabelEncoder()

In [18]:
list(le.classes_)

['Arabic',
 'Chinese',
 'Dutch',
 'English',
 'Estonian',
 'French',
 'Hindi',
 'Indonesian',
 'Japanese',
 'Korean',
 'Latin',
 'Persian',
 'Portugese',
 'Pushto',
 'Romanian',
 'Russian',
 'Spanish',
 'Swedish',
 'Tamil',
 'Thai',
 'Turkish',
 'Urdu']

In [19]:
Y2 = le.fit_transform(Y)

In [20]:
total_languages = df['language'].nunique()

In [21]:
Y2 = keras.utils.to_categorical(Y2,num_classes=total_languages)

In [22]:
np.shape(Y2)

(22000, 22)

In [23]:
X_train,X_test,Y_train,Y_test = train_test_split(train_df,Y2)

In [24]:
embedding_dims = 500
vocab_size = len(tokenizer.word_index)+1

In [25]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim = vocab_size,output_dim = embedding_dims,input_length = max_len),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(total_languages,activation=tf.nn.softmax)
                            ])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 500)          136984000 
_________________________________________________________________
flatten (Flatten)            (None, 50000)             0         
_________________________________________________________________
dense (Dense)                (None, 22)                1100022   
Total params: 138,084,022
Trainable params: 138,084,022
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer ='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])


In [28]:
model.fit(np.array(X_train),np.array(Y_train),epochs=5)


Train on 16500 samples
Epoch 1/5
16500/16500 [==============================] - 515s 31ms/sample - loss: 0.6298 - accuracy: 0.8315
Epoch 2/5
16500/16500 [==============================] - 523s 32ms/sample - loss: 0.1179 - accuracy: 0.9476
Epoch 3/5
16500/16500 [==============================] - 527s 32ms/sample - loss: 0.0901 - accuracy: 0.9535
Epoch 4/5
16500/16500 [==============================] - 527s 32ms/sample - loss: 0.0858 - accuracy: 0.9553
Epoch 5/5
16500/16500 [==============================] - 558s 34ms/sample - loss: 0.0828 - accuracy: 0.9563


In [29]:
model.evaluate(np.array(X_test),np.array(Y_test))


5500/5500 [==============================] - 1s 135us/sample - loss: 0.1668 - accuracy: 0.9342


[0.1668202197382396, 0.9341818]

In [30]:
print("English ",le.transform(['English']))
print("French ",le.transform(['French']))
print("Dutch ",le.transform(['Dutch']))
print("Swedish ",le.transform(['Swedish']))

English  [3]
French  [5]
Dutch  [2]
Swedish  [17]


In [31]:
#text = ["Once you know all the elements, it's not difficult to pull together a sentence."]
#text = ["När du väl känner till alla element är det inte svårt att ta ihop en mening."] #swedish
text = ["Als je eenmaal alle elementen kent, is het niet moeilijk om een zin samen te stellen."] # Dutch
#text =["Une fois que vous connaissez tous les éléments, il n'est pas difficile de rassembler une phrase."] #French

In [32]:
test_text = tokenizer.texts_to_sequences(text)
test_text = tf.keras.preprocessing.sequence.pad_sequences(test_text, maxlen=max_len)

In [33]:
predictions = model.predict(test_text)

In [34]:
out = predictions.argmax()
print(le.inverse_transform([out]))
print(predictions)

['Dutch']
[[6.1012186e-05 5.6895602e-04 9.9741167e-01 1.7087336e-04 3.1848587e-05
  4.1141186e-05 4.3521188e-05 4.6991114e-05 2.1377979e-04 8.8911162e-05
  2.1953195e-04 4.7906600e-05 7.8340156e-05 8.9330599e-05 1.4844121e-04
  7.7085897e-05 6.5119493e-05 9.8221877e-05 2.2798829e-05 1.2683671e-04
  3.2161977e-04 2.6153473e-05]]


In [35]:
text2 =["Une fois que vous connaissez tous les éléments, il n'est pas difficile de rassembler une phrase."] #French

In [36]:
test_text2 = tokenizer.texts_to_sequences(text2)
test_text2 = tf.keras.preprocessing.sequence.pad_sequences(test_text2, maxlen=max_len)

In [37]:
predictions2 = model.predict(test_text2)

In [38]:
out2 = predictions2.argmax()
print(le.inverse_transform([out2]))
print(predictions2)

['French']
[[8.0978347e-04 1.0693547e-03 1.4357541e-02 3.5939406e-04 2.0476719e-03
  9.3291909e-01 5.0980627e-04 8.1723952e-04 1.0252260e-02 1.7609925e-03
  5.9154304e-03 7.0889835e-04 4.5079188e-03 8.1163947e-04 4.3656286e-03
  7.6233491e-04 9.9584563e-03 8.2183664e-04 2.0575712e-03 3.3320291e-03
  1.1348643e-03 7.2022399e-04]]
